<a href="https://colab.research.google.com/github/Tanvii26/Store-Demand-Forecast/blob/main/Store_Demand_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q upgini catboost #-q = quiet (suppress output msgs during installation)

Upgini-> Find and add relevant features to ML pipeline from external/public data souces. Greater features enhance model accuracy & robustness

CatBoost-> Algo for gradient boosting on decision trees. 


*   Gradient Boost - Weak learners are sequentially trained to correct mistakes from previous models.
*   Decision Tree - predicts based on if-else conditions. Gradient boost on decision tree builds an ensemble of trees.

Each tree is trained to improve prev.



In [ ]:
import pandas as pd

df = pd.read_csv('/content/train.csv')

In [ ]:
df.head()

,date,store,item,sales
0,2013-01-01,1.0,1.0,13.0
1,2013-01-02,1.0,1.0,11.0
2,2013-01-03,1.0,1.0,14.0
3,2013-01-04,1.0,1.0,13.0
4,2013-01-05,1.0,1.0,10.0


In [ ]:
df['store'] = df['store'].astype(str)
df['item'] = df['item'].astype(str)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.sort_values('date', inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.head()

,date,store,item,sales
0,2013-01-01,1.0,1.0,13.0
1,2013-01-01,10.0,9.0,23.0
2,2013-01-01,7.0,5.0,5.0
3,2013-01-01,3.0,8.0,54.0
4,2013-01-01,4.0,7.0,26.0


In [ ]:
df.dtypes

date     datetime64[ns]
store            object
item             object
sales           float64
dtype: object

In [ ]:
# train- 2013 to 2016
# test = 2017 data

train=df[df['date']<"2017-01-01"]

test=df[df['date']>= "2017-01-01"]

In [ ]:
train_features = train.drop(columns=["sales"])
train_target = train['sales']

In [ ]:
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

Enrich features (upgini)

In [ ]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys={
        "date": SearchKey.DATE,
    },
    cv = CVType.time_series
)

In [ ]:
enricher.fit(train_features,train_target, eval_set=[(test_features, test_target)])
# shap as imp of features inorder

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

Column name,Status,Errors
country_code,All valid,-
date,All valid,-
target,All valid,-



Running search request, search_id=4a95204c-ab7f-4b5b-996e-e77182870696
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


32 relevant feature(s) found with the search keys: ['date', 'country_code']


,,item,0.630688,100.000000,categorical,
,,store,0.185218,100.000000,categorical,
Upgini,Public data,f_weather_date_weather_pca_0_d7e0a1fc,0.036801,100.000000,numerical,Free
Upgini,Public data,f_events_country_date_weekday_9c7b0a2a,0.029645,100.000000,numerical,Free
Upgini,Public data,f_events_date_year_cos1_9014a856,0.019381,100.000000,numerical,Free
Upgini,Public data,f_events_date_week_sin1_847b5db1,0.015857,100.000000,numerical,Free
Upgini,Public data,f_events_date_week_cos3_7525fe31,0.013647,100.000000,numerical,Free
Upgini,Public data,f_events_country_date_holiday_state_legal_84d_ahead_f83d666e,0.010619,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_31_fa6d9a99,0.010505,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_48_b39cd0c4,0.008400,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_34_c3ef5b4f,0.006889,100.000000,numerical,Free


We detected 609 outliers in your sample.
Examples of outliers with maximum value of target:
263    181.0
535    178.0
193    177.0
Name: target, dtype: float64
Outliers will be excluded during the metrics calculation.
Before dropping target outliers size: 173153
After dropping target outliers size: 172544
Calculating accuracy uplift after enrichment...
which makes metrics between the train and eval_set incomparable.

Quality metrics


,Rows,Baseline mean_squared_error,Enriched mean_squared_error,Uplift
,,,,
Train,138490,213.409498,86.115911,127.293587
Eval 1,34054,356.829301,229.831601,126.997699


In [ ]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

In [ ]:
model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

In [ ]:
enriched_train_features = enricher.transform(train_features, keep_input=True)
enriched_test_features = enricher.transform(test_features, keep_input=True)

In [ ]:
model.fit(enriched_train_features, train_target)

pred = model.predict(enriched_test_features)
eval_metric(test_target.values, pred, "SMAPE")

[14.833930339659467]

In [ ]:
pred.shape

(34358,)

In [ ]:
# store predictions
predictions = pd.DataFrame(pred)
sub_df = pd.read_csv('/content/sample_submission.csv')
datasets=pd.concat([sub_df['id'],predictions], axis=1)
datasets.columns=['id','sales']
datasets.to_csv('/content/sample_submission.csv', index=False)